Experiment 3: Vary the number of trainning examples
    training exampless = 15000 , 30000

In [1]:
# Import the necessary libraries
import numpy as np
import math
from scipy.special import expit
from sklearn.metrics import confusion_matrix, accuracy_score 
import csv
import random
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load the training data
train_file = open("mnist_train.csv","r")
train_data = csv.reader(train_file)
train = np.array(list(train_data))
#train

In [ ]:
# Load the test data
test_file = open("mnist_test.csv","r")
test_data = csv.reader(test_file)
test = np.array(list(test_data))
test

In [ ]:
np.random.shuffle(train)
#The tarining examples vary, so consider 15000, 30000
train = train[0:15000]

In [ ]:
# Initialise the values of bias, learning rate and momentum
bias = 1
learning_rate = 0.1
m = 0.9

In [ ]:
# Number of hidden inputs
n = 100

In [ ]:
# Applying the weights
weight_ih = np.random.uniform(-0.05,0.05,(785,n))
#print(weight_ih.shape)
weight_h0 = np.random.uniform(-0.05,0.05,(n+1,10)) 
#print(weight_h0.shape)

# store previous delta weight from hidden to output layer
previous_weight_h0 = np.zeros((n+1,10))
# store previous delta weight from input to hidden layer
previous_weight_ih = np.zeros((785,n))

In [ ]:
# matrix to store the activation h1...hk
hl_input = np.zeros((1,n+1))
hl_input[0,0] = 1

In [ ]:
# Define the perceptron
def multi_perceptron(epoch,input_ds,set_flag):
    global weight_ih,weight_h0,previous_weight_ih,previous_weight_h0 
    predict_list = []
    actual_list = []
    for i in range(input_ds.shape[0]):
        target_class = input_ds[i,0].astype('int')
        actual_list.append(target_class)
        xi = input_ds[i].astype('float16')/255
        xi[0] = bias
        xi = xi.reshape(1,785)
        z_hl = np.dot(xi,weight_ih) 
        sig_hl = expit(z_hl)
        # print(hl_input.shape)
        z_ol = np.dot(hl_input,weight_h0) 
        sig_ol = expit(z_ol)
        # print(sig_ol)
        predict = np.argmax(sig_ol)
        # print(sig_ol.shape)
        if epoch>0 and set_flag == 1:
            ##error term for output unit
            tk = np.zeros((1,10))+0.1
            tk[0,target_class] = 0.9
            # print(tk)
            error_ol = sig_ol*(1-sig_ol)* (tk - sig_ol)
            # print("error_ol shape for ",epoch," ",error_ol.shape)
            
            #error term for hidden unit
            error_hl = sig_hl*(1-sig_hl)* np.dot (error_ol,weight_h0[1:,:].T) 
            # print("error_hl shape for ",epoch," ",error_hl.shape)
            
            # Update the weights #
            delta_weight_h0 = (learning_rate * error_ol * hl_input.T) + (m * previous_weight_h0) 
            previous_weight_h0 = delta_weight_h0
            weight_h0 = weight_h0 + delta_weight_h0
            
            ### Input to output layer wt updation
            delta_weight_ih = (learning_rate * error_hl * xi.T) + (m * previous_weight_ih) 
            previous_weight_ih = delta_weight_ih
            weight_ih = weight_ih + delta_weight_ih

    accur = (np.array(predict_list) == np.array(actual_list)).sum()
    accuracy = accur_sum/float(len(actual_list))*100
    print("len of actual_list after ", epoch," is ",len(actual_list))
    print("len of predict_list after ", epoch," is ",len(predict_list))
    
    if(set_flag == 0):
        print("Confusion matrix for epoch ",epoch)  
        print(confusion_matrix(actual_list,predict_list))
    return accuracy      

In [ ]:
def store_accuracy(accuract_index,accuracy,input_ds):
     with open(input_ds, 'a', newline='') as myfile:
        wr = csv.writer(myfile)
        wr.writerow([accuracy_index,accuracy])

In [ ]:
for each in range(50):
    train_accuracy = multi_perceptron(each,train,1)
    test_accuracy = multi_perceptron(each,test,0)
    store_accuracy(each,train_accuracy,'train_quarter'+str(m)+'.csv') 
    store_accuracy(each,test_accuracy,'test_quarter'+str(m)+'.csv'

In [ ]:
# Plot the graph
x1, y1 = np.loadtxt("train_quarter.csv",delimiter=',',unpack=True) 
x2, y2 = np.loadtxt("test_quarter.csv",delimiter=',',unpack=True) 
plt.plot(x1,y1, label="Training Set")
plt.plot(x2,y2, label="Test Set")
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%) ')
plt.legend()
plt.show()